### Read configuration values

In [ ]:
#!value --name "configValues" --from-file appsettings.json --mime-type application/json

In [ ]:
using System.Text.Json;
#!share configValues --from value --mime-type application/json

public class ConfigurationJson
{
    public string tenantId {get;set;}
    public string applicationIdConfidentialClient {get;set;}
    public string applicationSecretConfidentialClient {get;set;}
    public string applicationIdPublicClient {get;set;}
}

var json = JsonSerializer.Deserialize<ConfigurationJson>(configValues);

### HTTP Client .Net (application permissions)

In [ ]:
#r "nuget:Microsoft.Graph"
#r "nuget:Microsoft.Identity.Client"

using System;
using System.Collections.Generic;
using System.Threading;
using Microsoft.Graph;
using Microsoft.Identity.Client;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading.Tasks;


private static HttpClient GetAuthenticatedHTTPClient()
{
	var authenticationProvider = CreateAuthorizationProvider();
	_httpClient = new HttpClient(new AuthHandler(authenticationProvider, new HttpClientHandler()));
	return _httpClient;
}

private static IAuthenticationProvider CreateAuthorizationProvider()
{
	var clientId = json.applicationIdConfidentialClient;
	var clientSecret = json.applicationSecretConfidentialClient;
	var tenantId = json.tenantId;
	var authority = $"https://login.microsoftonline.com/{tenantId}/v2.0";

	//this specific scope means that application will default to what is defined in the application registration rather than using dynamic scopes
	List<string> scopes = new List<string>();
	scopes.Add("https://graph.microsoft.com/.default");

	var cca = ConfidentialClientApplicationBuilder.Create(clientId)
											.WithAuthority(authority)
											.WithClientSecret(clientSecret)
											.Build();
	return new MsalAuthenticationProvider(cca, scopes.ToArray());
}

// Define other methods and classes here
public class MsalAuthenticationProvider : IAuthenticationProvider
{
	private IConfidentialClientApplication _clientApplication;
	private string[] _scopes;

	public MsalAuthenticationProvider(IConfidentialClientApplication clientApplication, string[] scopes)
	{
		_clientApplication = clientApplication;
		_scopes = scopes;
	}

	/// <summary>
	/// Update HttpRequestMessage with credentials
	/// </summary>
	public async Task AuthenticateRequestAsync(HttpRequestMessage request)
	{
		var token = await GetTokenAsync();
		request.Headers.Authorization = new AuthenticationHeaderValue("bearer", token);
	}

	/// <summary>
	/// Acquire Token 
	/// </summary>
	public async Task<string> GetTokenAsync()
	{
		AuthenticationResult authResult = null;
		authResult = await _clientApplication.AcquireTokenForClient(_scopes)
							.ExecuteAsync();
		return authResult.AccessToken;
	}
}

public class AuthHandler : DelegatingHandler
{
	private IAuthenticationProvider _authenticationProvider;

	public AuthHandler(IAuthenticationProvider authenticationProvider, HttpMessageHandler innerHandler)
	{
		InnerHandler = innerHandler;
		_authenticationProvider = authenticationProvider;
	}

	protected override async Task<HttpResponseMessage> SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)
	{
		await _authenticationProvider.AuthenticateRequestAsync(request);
		return await base.SendAsync(request, cancellationToken);
	}
}


private static HttpClient _httpClient;

HttpClient httpClient = GetAuthenticatedHTTPClient();

// if prefer to get access token directly, uncomment below lines and the corresponding GetAccessToken() method and AuthResult class
//var accessToken = GetAccessToken();
//httpClient.DefaultRequestHeaders.Add("Authorization", $"bearer {accessToken}");

var version = "v1.0";
var graphRequestUrl = $"https://graph.microsoft.com/{version}/users?$top=1";

var result = httpClient.GetStringAsync(graphRequestUrl).Result;
result

Installed Packages Microsoft.Graph, 4.39.0 Microsoft.Identity.Client, 4.46.2

{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users","@odata.nextLink":"https://graph.microsoft.com/v1.0/users?$top=1&$skiptoken=RFNwdAIAAQAAACI6QWRhbXNATTM2NXg1NTA1MzYuT25NaWNyb3NvZnQuY29tKVVzZXJfZGYzMzJkNzAtNDU5OS00YTJkLWJkOTYtOWY0NDdhN2FhOTlkuQAAAAAAAAAAAAA","value":[{"businessPhones":[],"displayName":"Conf Room Adams","givenName":null,"jobTitle":null,"mail":"Adams@M365x550536.OnMicrosoft.com","mobilePhone":null,"officeLocation":null,"preferredLanguage":null,"surname":null,"userPrincipalName":"Adams@M365x550536.OnMicrosoft.com","id":"df332d70-4599-4a2d-bd96-9f447a7aa99d"}]}

### Microsoft Graph .Net SDK (delegated permissions, device code)

In [ ]:
#r "nuget:Microsoft.Graph"
#r "nuget:Microsoft.Identity.Client"
#r "nuget:Azure.Identity"

using Azure.Identity;
using Microsoft.Graph;
using Microsoft.Identity.Client;
using System.Threading;

var tenantId = json.tenantId;
var clientId = json.applicationIdPublicClient;

//this specific scope means that application will default to what is defined in the application registration rather than using dynamic scopes
var scopes = new [] {"https://graph.microsoft.com/.default"};

var options = new TokenCredentialOptions
{
    AuthorityHost = AzureAuthorityHosts.AzurePublicCloud
};

Func<DeviceCodeInfo, CancellationToken, Task> callback = (code, cancellation) => {
    Console.WriteLine(code.Message);
    return Task.FromResult(0);
};

var deviceCodeCredential = new DeviceCodeCredential(
    callback, tenantId, clientId, options);

var graphClient = new GraphServiceClient(deviceCodeCredential, scopes);

var result = graphClient.Users.Request().GetAsync().Result[0];
$"{result.DisplayName} | {result.UserPrincipalName} | {result.Id}"

Installed Packages Azure.Identity, 1.6.1 Microsoft.Graph, 4.40.0 microsoft.identity.client, 4.46.1

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ALUM4QSSV to authenticate.


Conf Room Adams | Adams@M365x550536.OnMicrosoft.com | df332d70-4599-4a2d-bd96-9f447a7aa99d

### Microsoft Graph .Net SDK (application permissions, client credentials)

In [ ]:
#r "nuget:Microsoft.Graph"
#r "nuget:Azure.Identity"

using Azure.Identity;
using Microsoft.Graph;

var tenantId = json.tenantId;
var clientId = json.applicationIdConfidentialClient;
var clientSecret = json.applicationSecretConfidentialClient;

//this specific scope means that application will default to what is defined in the application registration rather than using dynamic scopes
var scopes = new [] {"https://graph.microsoft.com/.default"};

var options = new TokenCredentialOptions
{
    AuthorityHost = AzureAuthorityHosts.AzurePublicCloud
};

var clientSecretCredential = new ClientSecretCredential(
    tenantId, clientId, clientSecret, options);

var graphClient = new GraphServiceClient(clientSecretCredential, scopes);

var result = graphClient.Users.Request().GetAsync().Result[0];
$"{result.DisplayName} | {result.UserPrincipalName} | {result.Id}"

Installed Packages Azure.Identity, 1.6.1 Microsoft.Graph, 4.40.0

Conf Room Adams | Adams@M365x550536.OnMicrosoft.com | df332d70-4599-4a2d-bd96-9f447a7aa99d